In [57]:
import sys
sys.path.append('..')

In [58]:
import CAT
import json
import torch
import logging
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter

In [59]:
def setuplogger():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(levelname)s %(asctime)s] %(message)s")
    handler.setFormatter(formatter)
    root.addHandler(handler)

In [60]:
setuplogger()

In [61]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

In [62]:
log_dir = f"../logs/{datetime.datetime.now().strftime('%Y-%m-%d-%H%M')}/"
print(log_dir)
writer = SummaryWriter(log_dir)

../logs/2024-09-29-1812/


In [63]:
# choose dataset here
dataset = 'assistment'
# modify config here
config = {
    'learning_rate': 0.0025,
    'batch_size': 64,
    'num_epochs': 8,
    'num_dim': 1, # for IRT or MIRT
    'device': 'cpu',
    # for NeuralCD
    'prednet_len1': 128,
    'prednet_len2': 64,
}
# fixed test length
test_length = 1
# choose strategies here
strategies = [CAT.strategy.RandomStrategy(), CAT.strategy.MFIStrategy(), CAT.strategy.KLIStrategy()]
# modify checkpoint path here
ckpt_path = './irt.pt'

In [64]:
# read datasets
test_triplets = pd.read_csv(f'./dataset/test_triples.csv', encoding='utf-8').to_records(index=False)
concept_map = json.load(open(f'./dataset/concept_map.json', 'r'))
concept_map = {int(k):v for k,v in concept_map.items()}
metadata = json.load(open(f'./dataset/metadata.json', 'r'))

In [65]:
test_data = CAT.dataset.AdapTestDataset(test_triplets, concept_map,
                                        metadata['num_test_students'], 
                                        metadata['num_questions'], 
                                        metadata['num_concepts'])

In [66]:
import warnings
warnings.filterwarnings("ignore")

In [67]:
for strategy in strategies:
    model = CAT.model.IRTModel(**config)
    model.init_model(test_data)
    model.adaptest_load(ckpt_path)
    test_data.reset()
    
    logging.info('-----------')
    logging.info(f'start adaptive testing with {strategy.name} strategy')

    logging.info(f'Iteration 0')
    # evaluate models
    results = model.evaluate(test_data)
    for name, value in results.items():
        logging.info(f'{name}:{value}')
        
    for it in range(1, test_length + 1):
        logging.info(f'Iteration {it}')
        # select question
        selected_questions = strategy.adaptest_select(model, test_data)
        for student, question in selected_questions.items():
            test_data.apply_selection(student, question)
        # update models
        model.adaptest_update(test_data)
        # evaluate models
        results = model.evaluate(test_data)
        # log results
        for name, value in results.items():
            logging.info(f'{name}:{value}')
            writer.add_scalars(name, {strategy.name: value}, it)

[INFO 2024-09-29 18:12:36,119] -----------
[INFO 2024-09-29 18:12:36,119] -----------
[INFO 2024-09-29 18:12:36,119] -----------
[INFO 2024-09-29 18:12:36,119] -----------
[INFO 2024-09-29 18:12:36,119] -----------
[INFO 2024-09-29 18:12:36,119] -----------
[INFO 2024-09-29 18:12:36,123] start adaptive testing with Random Select Strategy strategy
[INFO 2024-09-29 18:12:36,123] start adaptive testing with Random Select Strategy strategy
[INFO 2024-09-29 18:12:36,123] start adaptive testing with Random Select Strategy strategy
[INFO 2024-09-29 18:12:36,123] start adaptive testing with Random Select Strategy strategy
[INFO 2024-09-29 18:12:36,123] start adaptive testing with Random Select Strategy strategy
[INFO 2024-09-29 18:12:36,123] start adaptive testing with Random Select Strategy strategy
[INFO 2024-09-29 18:12:36,128] Iteration 0
[INFO 2024-09-29 18:12:36,128] Iteration 0
[INFO 2024-09-29 18:12:36,128] Iteration 0
[INFO 2024-09-29 18:12:36,128] Iteration 0
[INFO 2024-09-29 18:12:3